# **# Modeli Hugging Face Hub'dan İndirme ve Yükleme

Aşağıdaki kod, Hugging Face Hub'daki kendi modelinizi otomatik indirip yükler.  
MODEL_HUB_ID değişkenini kendi model kimliğinizle güncellediğinizden emin olun.  
Çalıştırdığınızda model cihazınıza (GPU veya CPU) yüklenir ve kullanıma hazır olur.
**

In [16]:
#@title 1. Gerekli Kütüphaneleri Kurma ve İçe Aktarma
# Bu hücre, gerekli kütüphaneleri kurar ve içe aktarır.
# Her Colab oturumunda bir kez çalıştırılması yeterlidir.
!pip install torch transformers huggingface_hub accelerate -q
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import ipywidgets as widgets # Daha interaktif prompt girişi için (opsiyonel)
from IPython.display import display, Markdown # Markdown formatında çıktı için
from transformers import pipeline
print("Kütüphaneler başarıyla yüklendi/içe aktarıldı.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.8 MB/s eta 0:00:00
Kütüphaneler başarıyla yüklendi/içe aktarıldı.


In [31]:
#@title Hugging Face Girişi (Gerekirse)
# Bu hücre, Hugging Face hesabınıza Colab üzerinden giriş yapmanızı sağlar.
# Özellikle private modellere erişmek veya Hub'a bir şeyler yüklemek için gereklidir.
# Public modelleri indirmek için genellikle bu adıma GEREK YOKTUR.

from huggingface_hub import notebook_login

# notebook_login() fonksiyonunu çağırarak giriş arayüzünü başlatın.
# Çalıştırdığınızda, bir metin kutusu çıkacak ve oraya Hugging Face Access Token'ınızı girmeniz istenecek.
# Token'ınızı https://huggingface.co/settings/tokens adresinden oluşturabilirsiniz (write yetkisiyle).
print("Hugging Face giriş arayüzü başlatılıyor...")
print("Aşağıdaki linke tıklayarak veya token'ınızı doğrudan girerek giriş yapabilirsiniz.")
notebook_login()

Hugging Face giriş arayüzü başlatılıyor...
Aşağıdaki linke tıklayarak veya token'ınızı doğrudan girerek giriş yapabilirsiniz.


SyntaxError: invalid syntax (<ipython-input-32-643b0cd071fe>, line 1)

In [37]:

import torch
#@title 2. Model ve Test Yapılandırması
# LÜTFEN BU HÜCREDEKİ MODEL_HUB_ID'Yİ KENDİ MODELİNİZLE GÜNCELLEYİN!

# Hugging Face Hub'daki modelinizin tam adı (kullanıcı_adı/model_adı)
MODEL_HUB_ID = "Emirhan05/distilgpt2-poetry-generation-ai"  # <<<--- BU SATIRI KENDİ MODEL KİMLİĞİNİZLE DEĞİŞTİRİN!

# Test parametreleri (kullanıcı arayüzünde de değiştirilebilir olacak)
DEFAULT_PROMPT = "The stars shine bright in the velvet night"
DEFAULT_MAX_LEN = 120
DEFAULT_NUM_POEMS = 1
DEFAULT_TEMPERATURE = 0.75
DEFAULT_TOP_K = 50
DEFAULT_TOP_P = 0.92
DEFAULT_NO_REPEAT_NGRAM_SIZE = 2

# Cihaz seçimi (Colab'da GPU varsa otomatik kullanır)
DEVICE_ID = 0 if torch.cuda.is_available() else -1
DEVICE_NAME = "GPU" if DEVICE_ID == 0 else "CPU"

print(f"Yapılandırma tamamlandı. Model ID: {MODEL_HUB_ID}")
print(f"Kullanılacak Cihaz: {DEVICE_NAME}")

# Model ID'sinin güncellenip güncellenmediğini kontrol et
if MODEL_HUB_ID == "distilgpt2-poetry-generation-ai":
    display(Markdown("<font color='red'>**UYARI:** `MODEL_HUB_ID` değişkenini kendi Hugging Face model kimliğinizle güncellemediniz! Lütfen yukarıdaki hücrede bu değişikliği yapın.</font>"))
else:
    print(f"Model Hugging Face Hub'dan yükleniyor: {MODEL_HUB_ID}...")
    try:
        text_generator_pipeline = pipeline(
            "text-generation",
            model=MODEL_HUB_ID,
            device=DEVICE_ID
        )
        print("Model ve tokenizer başarıyla Hub'dan yüklendi.")
    except Exception as e:
        print(f"HATA: Model Hub'dan yüklenirken bir sorun oluştu: {e}")
        print("Lütfen model kimliğinin doğru olduğundan, modelin public olduğundan (veya Colab'ın erişimi olduğundan)")
        print("ve internet bağlantınızın olduğundan emin olun.")
        text_generator_pipeline = None

Yapılandırma tamamlandı. Model ID: Emirhan05/distilgpt2-poetry-generation-ai
Kullanılacak Cihaz: GPU
Model Hugging Face Hub'dan yükleniyor: Emirhan05/distilgpt2-poetry-generation-ai...


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/528 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Device set to use cuda:0


Model ve tokenizer başarıyla Hub'dan yüklendi.


In [41]:
#@title 3. Modeli Hugging Face Hub'dan Yükleme
# Bu hücre, modeli ve tokenizer'ı Hugging Face Hub'dan yükler.
# Model büyükse veya internet yavaşsa bu işlem biraz zaman alabilir.

text_generator_pipeline = None # Başlangıçta None

if MODEL_HUB_ID == "distilgpt2-poetry-generation-ai":
    display(Markdown("<font color='red'>Model ID güncellenmediği için model yükleme adımı atlanıyor. Lütfen bir önceki hücrede `MODEL_HUB_ID`'yi güncelleyin.</font>"))
else:
    print(f"Model Hugging Face Hub'dan yükleniyor: {MODEL_HUB_ID}...")
    try:
        text_generator_pipeline = pipeline(
            "text-generation",
            model=MODEL_HUB_ID,
            device=DEVICE_ID
        )
        print("Model ve tokenizer başarıyla Hub'dan yüklendi.")
    except Exception as e:
        print(f"HATA: Model Hub'dan yüklenirken bir sorun oluştu: {e}")
        print("Lütfen model kimliğinin doğru olduğundan, modelin public olduğundan (veya Colab'ın erişimi olduğundan)")
        print("ve internet bağlantınızın olduğundan emin olun.")
        text_generator_pipeline = None


Model Hugging Face Hub'dan yükleniyor: Emirhan05/distilgpt2-poetry-generation-ai...


Device set to use cuda:0


Model ve tokenizer başarıyla Hub'dan yüklendi.


In [42]:
#@title 4. Şiir Üretme Fonksiyonu
def generate_poem_colab(
    prompt,
    max_len,
    num_poems,
    temperature,
    top_k,
    top_p,
    no_repeat_ngram
):
    if text_generator_pipeline is None:
        return "HATA: Model yüklenemedi. Lütfen bir önceki 'Modeli Yükleme' hücresini kontrol edin."

    if not prompt or not prompt.strip():
        return "Lütfen bir başlangıç dizesi veya tema girin."

    try:
        print(f"Şiir üretiliyor... Prompt: '{prompt}'")
        pad_token_id_to_use = text_generator_pipeline.tokenizer.eos_token_id if \
                              hasattr(text_generator_pipeline, 'tokenizer') and \
                              text_generator_pipeline.tokenizer.eos_token_id is not None else 50256

        generated_outputs = text_generator_pipeline(
            prompt,
            max_length=int(max_len),
            num_return_sequences=int(num_poems),
            do_sample=True,
            temperature=float(temperature),
            top_k=int(top_k),
            top_p=float(top_p),
            no_repeat_ngram_size=int(no_repeat_ngram),
            pad_token_id=pad_token_id_to_use,
            truncation=True
        )

        poems_output_md = "" # Markdown formatında çıktı için
        for i, output in enumerate(generated_outputs):
            if 'generated_text' in output:
                poems_output_md += f"### Üretilen Şiir {i+1} (HUB Modelinden)\n"
                poems_output_md += f"```\n{output['generated_text']}\n```\n" # Şiiri code block içinde göster
            else:
                poems_output_md += f"<font color='red'>HATA: Şiir {i+1} üretilirken 'generated_text' anahtarı bulunamadı.</font>\n"

        print("Şiir üretimi tamamlandı.")
        return poems_output_md

    except Exception as e:
        print(f"HATA: Şiir üretimi sırasında bir sorun oluştu: {e}")
        return f"<font color='red'>Bir hata oluştu: {str(e)}</font>"

In [43]:
import ipywidgets as widgets # Bu satır zaten ilk hücrede var, ancak bağımsız çalıştırmak için buraya da eklenebilir.
from IPython.display import display, Markdown
# Girdi elemanları
prompt_input = widgets.Text(
    value=DEFAULT_PROMPT,
    placeholder='Şiir için bir başlangıç dizesi girin (İngilizce)...',
    description='Prompt (İngilizce):',
    layout=widgets.Layout(width='90%')
)

max_len_slider = widgets.IntSlider(
    value=DEFAULT_MAX_LEN, min=50, max=350, step=10, description='Max Uzunluk:', layout=widgets.Layout(width='50%'), style = {'description_width': 'initial'}
)
num_poems_slider = widgets.IntSlider(
    value=DEFAULT_NUM_POEMS, min=1, max=3, step=1, description='Şiir Sayısı:', layout=widgets.Layout(width='50%'), style = {'description_width': 'initial'}
)
temperature_slider = widgets.FloatSlider(
    value=DEFAULT_TEMPERATURE, min=0.1, max=1.5, step=0.05, description='Sıcaklık:', layout=widgets.Layout(width='50%'), style = {'description_width': 'initial'}
)
top_k_slider = widgets.IntSlider(
    value=DEFAULT_TOP_K, min=1, max=100, step=1, description='Top-K:', layout=widgets.Layout(width='50%'), style = {'description_width': 'initial'}
)
top_p_slider = widgets.FloatSlider(
    value=DEFAULT_TOP_P, min=0.1, max=1.0, step=0.01, description='Top-P:', layout=widgets.Layout(width='50%'), style = {'description_width': 'initial'}
)
ngram_slider = widgets.IntSlider(
    value=DEFAULT_NO_REPEAT_NGRAM_SIZE, min=0, max=5, step=1, description='N-gram Engelleme:', layout=widgets.Layout(width='50%'), style = {'description_width': 'initial'}
)

button = widgets.Button(description="Şiir Üret!")
output_area = widgets.Output() # Çıktıyı burada göstereceğiz

def on_button_clicked(b):
    with output_area:
        output_area.clear_output(wait=True) # Önceki çıktıyı temizle
        if text_generator_pipeline is None and not (MODEL_HUB_ID == "emirhan/distilgpt2-poetry-emirhan" or "SENIN_HUGGINGFACE_KULLANICI_ADIN" in MODEL_HUB_ID):
             display(Markdown("<font color='red'>**HATA:** Model yüklenemedi. Lütfen '3. Modeli Hugging Face Hub'dan Yükleme' hücresini çalıştırdığınızdan ve başarılı olduğundan emin olun.</font>"))
             return
        elif text_generator_pipeline is None:
             display(Markdown("<font color='red'>**HATA:** Model yüklenemedi. Lütfen '2. Model ve Test Yapılandırması' hücresinde `MODEL_HUB_ID`'yi güncelleyip '3. Modeli Yükleme' hücresini tekrar çalıştırın.</font>"))
             return


        generated_poetry_md = generate_poem_colab(
            prompt_input.value,
            max_len_slider.value,
            num_poems_slider.value,
            temperature_slider.value,
            top_k_slider.value,
            top_p_slider.value,
            ngram_slider.value
        )
        display(Markdown(generated_poetry_md))

button.on_click(on_button_clicked)

# Arayüzü göster
display(Markdown("## 📜 Şiir Üretici Test Arayüzü 📜"))
display(Markdown("Aşağıdaki alana İngilizce bir başlangıç dizesi girin ve parametreleri ayarladıktan sonra 'Şiir Üret!' butonuna tıklayın."))
display(prompt_input)
display(widgets.HBox([max_len_slider, num_poems_slider]))
display(widgets.HBox([temperature_slider, top_k_slider]))
display(widgets.HBox([top_p_slider, ngram_slider]))
display(button)
display(output_area)

## 📜 Şiir Üretici Test Arayüzü 📜

Aşağıdaki alana İngilizce bir başlangıç dizesi girin ve parametreleri ayarladıktan sonra 'Şiir Üret!' butonuna tıklayın.

Text(value='The stars shine bright in the velvet night', description='Prompt (İngilizce):', layout=Layout(widt…

Button(description='Şiir Üret!', style=ButtonStyle())

Output()